# Proxy Rotation (Note: Use EliteProxies)

In [8]:
import requests
from bs4 import BeautifulSoup

In [3]:
res = requests.get('https://free-proxy-list.net')
content = BeautifulSoup(res.text, 'lxml')
table = content.find('table')
rows = table.find_all('tr')
cols = [[col.text for col in row.find_all('td')] for row in rows]

proxies = []
proxy_index = 0

for col in cols:
    try:
        if col[4] == 'elite proxy' and col[6] == 'yes':
            proxies.append('https://' + col[0] + ':' + col[1])
    except:
        pass
# print (proxies)

def fetch(url, params):
    global proxy_index
    
    while proxy_index < len(proxies):
        try:
            print('Trying proxy:', proxies[proxy_index])
            res = requests.get(url, proxies={'https': proxies[proxy_index]}, params=params, timeout=5)
            return res
            
        except:
            print('Bad proxy...')
            proxy_index += 1

for page in range(0, 4):
    params = {'page': page}
    res = fetch('https://www.google.com/', params=params)
    proxy_index += 1
    print('ip', res.json()['ip'])
    print('url:', res.json()['url'])  

Trying proxy: https://47.240.160.90:10001
Bad proxy...
Trying proxy: https://195.158.30.232:3128
Bad proxy...
Trying proxy: https://5.161.105.105:80
Bad proxy...
Trying proxy: https://43.225.23.132:80
Bad proxy...
Trying proxy: https://150.136.108.121:3128
Bad proxy...
Trying proxy: https://85.25.91.141:15333
Bad proxy...
Trying proxy: https://85.195.104.71:80
Bad proxy...
Trying proxy: https://38.108.119.176:59394
Bad proxy...
Trying proxy: https://50.231.95.3:8080
Bad proxy...
Trying proxy: https://18.230.58.67:3128
Bad proxy...
Trying proxy: https://78.46.123.202:80
Bad proxy...
Trying proxy: https://5.189.184.6:80
Bad proxy...
Trying proxy: https://1.10.141.220:54620
Bad proxy...
Trying proxy: https://154.66.210.1:8080
Bad proxy...
Trying proxy: https://151.106.17.122:1080
Bad proxy...
Trying proxy: https://151.106.17.125:1080
Bad proxy...
Trying proxy: https://151.106.17.123:1080
Bad proxy...


AttributeError: 'NoneType' object has no attribute 'json'

# ------ OpenLibrary.org (below)------

# Word Combinations (of 20 Words Given)

In [31]:
import requests
import time
import requests
from bs4 import BeautifulSoup

In [25]:
words2Scrap=["Table", "Fog", "Wikipedia", "Empire", "Ruin", "Era", "English", "Library", "Twin", "Tower", "Book", "Art",
             "Science", "Poor", "Rich", "Dad", "Heart", "Power", "Medicine", "Borrow" ]

In [26]:
from itertools import combinations

#Source: https://stackoverflow.com/questions/464864/how-to-get-all-possible-combinations-of-a-list-s-elements#answer-32555776
def combs(x):
    return [c for i in range(len(x)+1) for c in combinations(x,i)]

In [27]:
lister= list(combs(words2Scrap))

In [28]:
len(lister)

1048576

In [29]:
def formatSearchTerm4url(typle,index):
    searchTerm=""
    for i,d in enumerate(list(typle[index])):
        if (i!=0):
            searchTerm+="+"+str(d)
        else:
            searchTerm=d
    urlMain= f"https://openlibrary.org/search?q=paper+trains&mode=everything"
    return urlMain


In [30]:
def getRobotDelay(num_runs,delayTime):
    #every 10 runs/pages or if first time, get delay from robots.txt
    if (num_runs%20==0 or delayTime==0):
        res = requests.get('https://openlibrary.org/robots.txt')
        content = BeautifulSoup(res.text, 'lxml')

        delayTime = float(str(content).split("\nCrawl-delay: ")[-1].split("\n")[0])
        print("just got robot.txt... current delay time: ",delayTime)
        return delayTime
    else:
        return delayTime
        

In [31]:
import requests
from bs4 import BeautifulSoup
import time
import numpy

def getLinksOfPageURL(lister, index,urls, num_runs, delayTime):
    delayTime = getRobotDelay(num_runs,delayTime)
    
    if (type(lister) is not str):
        URL =formatSearchTerm4url(lister,index)
        page = requests.get(URL)

        soup = BeautifulSoup(page.content, "html.parser")
        job_elements = soup.find(class_="pagination")

        page_no=0

        if (lister[index]!=()):
            pages= job_elements.find_all(class_="ChoosePage",href=True)

            print(f"\n\n\n\n\n\nURL of Index {index}...")
            for indexPage,data in enumerate(pages):
                if (indexPage==0):
                    #then add page 1 (since there's page 2)
                    url="https://openlibrary.org"+data['href'][:len(data['href'])-1]+"1"
                    page_no=1
                    print ("Found URL:", (url))
                    urls.append(url)
                    
                    #starts with page 2 thus add this too
                    url="https://openlibrary.org"+data['href']
                    print ("Found URL:",url )
                    page_no=int(data.text)
                    urls.append(url)
                    
                else:
                    try:
                        url="https://openlibrary.org"+data['href']
                        print ("Found URL:",url )
                        page_no=int(data.text)
                        urls.append(url)
                        
                        print(len(urls))
                    except:
                        pass
                    

            if (page_no!=0):
                url="https://openlibrary.org"+data['href'][:len(data['href'])-1]+str(page_no)
                print("zzzzz Sleeping for ",delayTime," sec(s), from Robot.txt")
                time.sleep(delayTime)
                print("ooooo Awake Now")
                
                getLinksOfPageURL(url,page_no,urls,int(page_no),delayTime)

        else:
            print(f"NOTE: Lister's Index of {index} has ZERO elements!")
            return []
    
    else:
        delayTime = getRobotDelay(num_runs,delayTime)
        page = requests.get(lister)
        
        print("Total Pages So Far: ", index)
        page_no=index

        try:
            page = requests.get(lister)
            soup = BeautifulSoup(page.content, "html.parser")
            job_elements = soup.find(class_="pagination")

            pages= job_elements.find_all(class_="ChoosePage",href=True)

            print(f"\nFrom Page {index}...")
            for indexPage,data in enumerate(pages):
                    if ("Next" in data.text or "First" in data.text or "Previous" in data.text):
                        pass
                    elif (int(data.text)>page_no):
                        url="https://openlibrary.org"+data['href']
                        print ("Found URL:",url )
                        urls.append(url)
                        print(len(urls))
                        page_no=int(data.text)

            print("zzzzz Sleeping for ",delayTime," sec(s), from Robot.txt")
            time.sleep(delayTime)
            print("ooooo Awake Now")
            
            getLinksOfPageURL(url,page_no,urls,int(page_no),delayTime)
        except:
            print("End Of the Line.... No More Next Page")
            print(urls)
            return urls
        return urls

In [32]:
formatSearchTerm4url(lister,0)

'https://openlibrary.org/search?q=paper+trains&mode=everything'

# Get Each Page's URL and store in CSV

In [ ]:
urlFile=[]
for i in range(0,len(lister)):
    #start with number of runs 0
    urlFile.append(getLinksOfPageURL(lister,i,urlFile,i,0))
    urlFile = [x for x in urlFile if x != []]
    print("TOTAL LENGTH OF URLS COLLECTED for keywords\'",lister[i],"\' :",len(urlFile))
    
    a = numpy.asarray(urlFile)
    numpy.savetxt(f"URLScrapedKeyWordsLinks.csv", a, delimiter=",", fmt='%s')
#     numpy.savetxt(f"URLScrapedKeyWords_WITH_{len(urlFile)}_links.csv", a, delimiter=",", fmt='%s')

        
#NOTE: total 2865 pages

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


just got robot.txt... current delay time:  0.5
NOTE: Lister's Index of 0 has ZERO elements!
TOTAL LENGTH OF URLS COLLECTED for keywords' () ' : 0
just got robot.txt... current delay time:  0.5






URL of Index 1...
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=1
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=2
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=3
3
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=4
4
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=5
5
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=6
6
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=7
7
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=8
8
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=9
9
Found URL: https://openlibrary.org/search?q=paper

# Get Each Page's URL from CSV

In [41]:
import csv
pageURLs = []
with open('URLScrapedKeyWordsLinks.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are\n{", ".join(row)}')
            pageURLs.append(row)
            line_count += 1
        else:
            print(f'{row[0]}')
            line_count += 1
            pageURLs.append(row)
    print(f'Processed {line_count} lines.')

Column names are
https://openlibrary.org/search?q=paper+trains&mode=everything&page=1
https://openlibrary.org/search?q=paper+trains&mode=everything&page=2
https://openlibrary.org/search?q=paper+trains&mode=everything&page=3
https://openlibrary.org/search?q=paper+trains&mode=everything&page=4
https://openlibrary.org/search?q=paper+trains&mode=everything&page=5
https://openlibrary.org/search?q=paper+trains&mode=everything&page=6
https://openlibrary.org/search?q=paper+trains&mode=everything&page=7
https://openlibrary.org/search?q=paper+trains&mode=everything&page=8
https://openlibrary.org/search?q=paper+trains&mode=everything&page=9
https://openlibrary.org/search?q=paper+trains&mode=everything&page=10
https://openlibrary.org/search?q=paper+trains&page=11&mode=everything
https://openlibrary.org/search?q=paper+trains&page=12&mode=everything
https://openlibrary.org/search?q=paper+trains&page=13&mode=everything
https://openlibrary.org/search?q=paper+trains&page=14&mode=everything
https://open

# Get Each Book's URL

In [132]:
delayTime = 0
bookURLs =[]
for index, item in enumerate(pageURLs):
    delayTime = getRobotDelay(index,delayTime)
    print("\n\nzzzzz Sleeping for ",delayTime," sec(s), from Robot.txt")
    time.sleep(delayTime)
    print("ooooo Awake Now")
            
    page = requests.get(pageURLs[index][0]) #url
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find('ul', class_='list-books')
    for item in results:
        if (item.find("h3") == -1):
            pass
        else:
            url = "https://openlibrary.org"+item.find("a")['href']
            print(url)
            bookURLs.append(url)
    if (index ==2):
        break

just got robot.txt... current delay time:  0.5


zzzzz Sleeping for  0.5  sec(s), from Robot.txt
ooooo Awake Now
https://openlibrary.org/works/OL6729940W
https://openlibrary.org/works/OL6729938W
https://openlibrary.org/works/OL6729935W
https://openlibrary.org/works/OL6729930W
https://openlibrary.org/works/OL6729928W
https://openlibrary.org/works/OL6729927W
https://openlibrary.org/works/OL6729909W
https://openlibrary.org/works/OL25294299W
https://openlibrary.org/works/OL24162515W?edition=ia%3Apapertrainechoes0000phei
https://openlibrary.org/works/OL6729934W
https://openlibrary.org/works/OL6729921W
https://openlibrary.org/works/OL6729933W
https://openlibrary.org/works/OL6729924W
https://openlibrary.org/works/OL18883167W
https://openlibrary.org/works/OL11929301W
https://openlibrary.org/works/OL18883166W
https://openlibrary.org/works/OL25547462W
https://openlibrary.org/works/OL6729929W
https://openlibrary.org/works/OL6729910W
https://openlibrary.org/works/OL6729916W


zzzzz Sleeping for  0

In [426]:
def getBookDetails(soup):
    dict1={}
    
    try:
        results = soup.find('div', class_='workDetails')
    except:
        pass
    
    #get Book Title
    try:
        dict1['title'] = results.find("h1",class_="work-title").text
    except:
        pass
    
    #get Author of Book URL
    try:
        dict1["authorURL"] = "https://openlibrary.org"+results.find("h2",class_="edition-byline").find("a")['href']
    except:
        pass
    
    #get Book Date
    try:
        dict1["date"] = results.find("div",  class_="smallest").text.split(" | ")[0]
    except:
        pass
    
    #Get Subject(s) Name, URL
    try:
        dictNest={}
        subjects = results.find("div",  class_="subjects-content").find("span",class_="clamp").find_all("a")
        for index, subject in enumerate(subjects):
            dictNest[index] = {"name":subject.text, "href": "https://openlibrary.org"+subject['href']}
        dict1["subject"]=dictNest
    except:
        pass
    
    #Get Published Date
    publish_date = soup.find("div",class_="edition-omniline-item").find("span").text
    dict1["publish_date"] = publish_date
    
    
    #Book Details Section
    sections = soup.find("div",  class_="edition-info").find_all("div",class_="section")
    for section in sections:
        try:
            published_in = section.findChild("p").text
            dict1['published_in'] = published_in.replace("\n          ","")
        except:
            pass

            dewy = section.findChild("dl")
        try:


            try:
                for index,item in enumerate(dewy.find_all("dt")):
                    if ("OCLC/WorldCat" in dewy.find_all("dt")[index].text):
                        dict1["oclc_worldcat"] = dewy.find_all("dd")[index].text.replace("\n","").replace(" ","")
                    if ("ISBN" in dewy.find_all("dt")[index].text):
                        dict1["isbn"] = dewy.find_all("dd")[index].text.replace("\n","").replace(" ","")
                    if ("Open Library" in dewy.find_all("dt")[index].text):
                        dict1["open_library_id"] = dewy.find_all("dd")[index].text.replace("\n","").replace(" ","")

                    if ("Dewey Decimal Class" in dewy.find_all("dt")[index].text):
                        dict1["dewy_decimal_class"] = dewy.findChild("dd").text.replace("\n","").replace(" ","")

                    if  ("Number of pages" in dewy.find_all("dt")[index].text):
                        dict1["num_of_pages"] = dewy.find_all("dd")[index].text.replace("\n","").replace(" ","").replace(";","")
                    if ("Pagination" in dewy.find_all("dt")[index].text):
                        dict1["pagination"] = dewy.find_all("dd")[index].text.replace(" ;\n\n","")

            except:
                pass
    
        except:
            pass
    

    
    return dict1

In [434]:
import json
for index, item in enumerate(bookURLs):

    delayTime = getRobotDelay(index,delayTime)
    
    print("\n\nzzzzz Sleeping for ",delayTime," sec(s), from Robot.txt")
    time.sleep(delayTime)
    print("ooooo Awake Now")
    
    page = requests.get(bookURLs[index]) #url
    soup = BeautifulSoup(page.content, "html.parser")
    
    csvHere = getBookDetails(soup)
    try:
        with open(bookURLs[index].split("https://openlibrary.org/works/")[1]+'.json', 'w') as outfile:
            json.dump(csvHere, outfile)
            
        print("Saved CSV file as\'",bookURLs[index].split("https://openlibrary.org/works/")[1]+'.json\'')
    except Exception as e:
        print("Couldn't export to CSV for Book (URL)"+bookURLs[index]+"\nStack Trace Here:\n")
        print(e)
        
    #break after 4 saves
    if (index == 3): break

just got robot.txt... current delay time:  0.5


zzzzz Sleeping for  0.5  sec(s), from Robot.txt
ooooo Awake Now
Saved CSV file as' OL6729940W.json'


zzzzz Sleeping for  0.5  sec(s), from Robot.txt
ooooo Awake Now
Saved CSV file as' OL6729938W.json'


zzzzz Sleeping for  0.5  sec(s), from Robot.txt
ooooo Awake Now
Saved CSV file as' OL6729935W.json'


zzzzz Sleeping for  0.5  sec(s), from Robot.txt
ooooo Awake Now
Saved CSV file as' OL6729930W.json'


# ------ Wiki.org (below)------

# (1 of 2) Wikipedia Crawler Setup (Below)

In [1]:
words2Scrap=["Computer", "Glasgow", "United", "Kingdom", "Library", "Fog", "Empires", "Doctor", "Hospital", "Bachelor", 
"Degree", "Internet", "Things", "Information", "Info", "Retrieval", "Retrieve", "Info", "Universe", "University"]

In [2]:
import requests
import time
import json
from bs4 import BeautifulSoup
from itertools import combinations
import time

#Source: https://stackoverflow.com/questions/464864/how-to-get-all-possible-combinations-of-a-list-s-elements#answer-32555776
def combs(x):
    return [c for i in range(len(x)+1) for c in combinations(x,i)]

In [3]:
lister1= list(combs(words2Scrap))
lister = []

In [4]:
#total Keyword Combinations
len(lister1)

1048576

In [5]:
for item in lister1:
    if (len(item) == 5 or len(item) < 6) and len(item) !=0 :
        lister.append(item)

In [6]:
#total Keyword Combinations for word Combinations up to 5 words
len(lister)

21699

In [7]:
#get Word(s) into Wikipedia Search URL
def formatSearchTerm4url(typle,index):
    searchTerm=""
    for i,d in enumerate(list(typle[index])):
        if (i!=0):
            searchTerm+="+"+str(d)
        else:
            searchTerm=d
    urlMain= f"https://en.wikipedia.org/w/index.php?search="+ searchTerm

    # get the redirected URL's destination (.url) A.K.A non-Search query URL
    urlMain = requests.get(urlMain).url
    return urlMain

In [8]:
#Time Delay Sleeping between Requests (to prevent Server Overload and IP-Ban)
def sleepingNow(delayTime):
    print("\n\nzzzzz Sleeping for ",delayTime," sec(s)")
    time.sleep(delayTime)
    print("ooooo Awake Now")

# (2 of 2) Wikipedia Crawler (Below)

In [ ]:
import signal
import sys
import requests
import time
import json
from bs4 import BeautifulSoup
from itertools import combinations
import time
import random

words2Scrap=["Computer", "Glasgow", "United", "Kingdom", "Library", "Fog", "Empires", "Doctor", "Hospital", "Bachelor", 
"Degree", "Internet", "Things", "Information", "Info", "Retrieval", "Retrieve", "Info", "Universe", "University"]

#Source: https://stackoverflow.com/questions/464864/how-to-get-all-possible-combinations-of-a-list-s-elements#answer-32555776
def combs(x):
    return [c for i in range(len(x)+1) for c in combinations(x,i)]


lister1= list(combs(words2Scrap))
lister = []

for item in lister1:
    if (len(item) == 5 or len(item) < 6) and len(item) !=0 :
        lister.append(item)

def sleepingNow():
    randSeconds = random.randint(1, 600)
    print("\n\nzzzzz Sleeping for ",randSeconds," sec(s)")
    time.sleep(randSeconds)
    print("ooooo Awake Now")

def formatSearchTerm4url(typle,index):
    searchTerm=""
    for i,d in enumerate(list(typle[index])):
        if (i!=0):
            searchTerm+="+"+str(d)
        else:
            searchTerm=d
    urlMain= f"https://en.wikipedia.org/w/index.php?search="+ searchTerm

    # get the redirected URL's destination (.url) A.K.A non-Search query URL
    urlMain = requests.get(urlMain).url
    return urlMain

new1stDiscoveredURLs={}
new2ndDiscoveredURLs={}
wikiArray=[]
addedJson={}

def signal_handler(sig, frame):
    #Finished Main Seeds (addedJson)
    if len(addedJson.keys())!=0:
        wikiArray.append(addedJson)
    
    #Finished 2nd Deep Link Crawl
    if len(new1stDiscoveredURLs.keys())!=0:
        wikiArray.append(new1stDiscoveredURLs)
    
    #Finished 3rd Deep Link Crawl
    if len(new2ndDiscoveredURLs.keys())!=0:
        wikiArray.append(new1stDiscoveredURLs)
    
    #Export Json to File
    try:
        wikiJson ={}
        for item in wikiArray:
            for key, value in item.items():
                wikiJson[key]=value
        
        with open('wikiFile.json', 'w') as outfile:
            json.dump(wikiJson, outfile)
        print("Exported to JSON for Wiki URLs as name: \'wikiFile.json\'")
    except Exception as e:
        print("Couldn't export to CSV for Wiki URLs... Stack Trace Here:\n")
        print(e)
    sys.exit(0)

def getLinksFromSearch(lister,index):
    #prepare for Control-C Keyboard Intterupt (get all existing Wiki data into Json)
    signal.signal(signal.SIGINT, signal_handler)
    
    for indexCurr,itemCurr in enumerate(lister):
        
        URL = formatSearchTerm4url(lister,indexCurr)
        print("\n\nCurrently Main Seed:"+str(indexCurr+1)," of ",len(lister))
        if (indexCurr<index+1):
            
            #if using Keyword Search
            sleepingNow()
            connectPassed=True
            try:
                page = requests.get(URL)
            except:
                connectPassed=False
            

            if (connectPassed == True):
                soup = BeautifulSoup(page.content, "html.parser")
                job_elements = soup.find("h1",{"id":"firstHeading"})
                contentTitle = job_elements.text

                if ("Search results" in job_elements.text):
                
                    print("Page is Wiki Search Page: "+URL)
                    #returned page is a search result page with multiple wiki pages
                    job_elements = soup.find("ul",{"class":"mw-search-results"}).find_all("a",href=True)

                    for index1,item1 in enumerate(job_elements):
                        url = "https://en.wikipedia.org"+item1['href']

                        found=False
                        for index,item in enumerate(wikiArray):
                            if (url in item or url in addedJson):
                                found = True

                        if (found==False):
                            contentTxt = ""

                            #if using Keyword Search
                            sleepingNow()
                            connectPassed=True
                            try:
                                page = requests.get(url)
                            except:
                                connectPassed=False


                            if (connectPassed == True):
                                soup = BeautifulSoup(page.content, "html.parser")
                                job_elements = soup.find("h1",{"id":"firstHeading"})
                                contentTitle = job_elements.text
                                job_elements = soup.find("div",{"id":"mw-content-text"}).find_all()
                                for item in job_elements:
                                    try:
                                        contentTxt = contentTxt + item.text
                                    except:
                                        pass

                                print("URL: "+page.url)
                                print("TITLE~\n"+contentTitle)
                                print("content~\n"+contentTxt)

                                addedJson[page.url]={"title":contentTitle,"content":contentTxt}
                                print("***Added URL:", page.url)

                                ####################################
                                #Get 2nd Deep Link from Main Seed
                                potentialLinks = soup.find("div",{"id":"mw-content-text"}).find_all("a", href=True)
                                for link in potentialLinks:
                                    currLink = link['href']
                                    failed = False

                                    if currLink[0] == "#":
                                        failed = True


                                    #if URL is not a reference within the Current Wiki Page
                                    if (failed == False):
                                        #Check if URL is in Store
                                        for index,item in enumerate(wikiArray):
                                            if (currLink in addedJson):
                                                failed = True
                                            jsonData = wikiArray[index]
                                            for key1 in jsonData.keys():
                                                if key1 in currLink:
                                                    failed = True
                                    if ("/wiki/" in currLink and "wikipedia.org" not in currLink):
                                        currLink = "https://en.wikipedia.org" + currLink

                                    if (("search=" in currLink or "wiki/File:" in currLink) and "wikipedia" in currLink):
                                        print("Potential New Discovered URL is a Wiki File/Search (not adding):   "+currLink)
                                    elif (failed==False and "wikipedia" in currLink):
                                        contentTxt = ""

                                        #if using Keyword Search
                                        sleepingNow()
                                        connectPassed=True
                                        try:
                                            page = requests.get(currLink)
                                        except:
                                            connectPassed=False

                                        if (connectPassed == True):
                                            soup = BeautifulSoup(page.content, "html.parser")
                                            job_elements = soup.find("h1",{"id":"firstHeading"})
                                            contentTitle = job_elements.text
                                            job_elements = soup.find("div",{"id":"mw-content-text"}).find_all()
                                            for item in job_elements:
                                                try:
                                                    contentTxt = contentTxt + item.text
                                                except:
                                                    pass

                                            print("URL: "+page.url)
                                            print("TITLE~\n"+contentTitle)
                                            print("content~\n"+contentTxt)

                                            new1stDiscoveredURLs[page.url]={"title":contentTitle,"content":contentTxt}
                                            print("***Added URL:", page.url)

                                        else:
                                            print("cannot connect to URL:"+currLink)
                                    else:
                                        print("Potential New Discovered URL found in Store (not adding):   "+currLink)

                                    print("\n\nCurrently at Main Seed (Depth 2):"+str(indexCurr+1)," of ",len(lister),"\n| Main Seed Link:"+URL)  

                            else:
                                print("cannot connect to URL:"+url)
                        else:
                            print("duplicated URL found in Store (not adding):   ",url)
    
    
                else:
                    print("Page is Wiki Details Page: "+URL)

                    found=False
                    for index,item in enumerate(wikiArray):
                        if (URL in item or URL in addedJson):
                            found = True

                    if (found==False):
                        contentTxt = ""
                        
                        #if using Keyword Search
                        sleepingNow()
                        connectPassed=True
                        try:
                            page = requests.get(URL)
                        except:
                            connectPassed=False


                        if (connectPassed == True):
                            soup = BeautifulSoup(page.content, "html.parser")
                            job_elements = soup.find("h1",{"id":"firstHeading"})
                            contentTitle = job_elements.text
                            job_elements = soup.find("div",{"id":"mw-content-text"}).find_all()
                            for item in job_elements:
                                try:
                                    contentTxt = contentTxt + item.text
                                except:
                                    pass

                            print("URL: "+page.url)
                            print("TITLE~\n"+contentTitle)
                            print("content~\n"+contentTxt)

                            addedJson[page.url]={"title":contentTitle,"content":contentTxt}
                            print("***Added URL:", page.url)

                            ####################################
                            #Get 2nd Deep Link from Main Seed
                            potentialLinks = soup.find("div",{"id":"mw-content-text"}).find_all("a", href=True)
                            for link in potentialLinks:
                                currLink = link['href']
                                failed = False


                                if currLink[0] == "#":
                                    failed = True


                                #if URL is not a reference within the Current Wiki Page
                                if (failed == False):
                                    #Check if URL is in Store
                                    for index,item in enumerate(wikiArray):
                                        if (currLink in addedJson):
                                            failed = True
                                        jsonData = wikiArray[index]
                                        for key1 in jsonData.keys():
                                            if key1 in currLink:
                                                failed = True
                                if ("/wiki/" in currLink and "wikipedia.org" not in currLink):
                                    currLink = "https://en.wikipedia.org" + currLink

                                if (("search=" in currLink or "wiki/File:" in currLink) and "wikipedia" in currLink):
                                    print("Potential New Discovered URL is a Wiki File/Search (not adding):   "+currLink)
                                elif (failed==False and "wikipedia" in currLink):
                                    contentTxt = ""
                                    
                                    #if using Keyword Search
                                    sleepingNow()
                                    connectPassed=True
                                    try:
                                        page = requests.get(currLink)
                                    except:
                                        connectPassed=False

                                    if (connectPassed == True):
                                        soup = BeautifulSoup(page.content, "html.parser")
                                        job_elements = soup.find("h1",{"id":"firstHeading"})
                                        contentTitle = job_elements.text
                                        job_elements = soup.find("div",{"id":"mw-content-text"}).find_all()
                                        for item in job_elements:
                                            try:
                                                contentTxt = contentTxt + item.text
                                            except:
                                                pass

                                        print("URL: "+page.url)
                                        print("TITLE~\n"+contentTitle)
                                        print("content~\n"+contentTxt)

                                        new1stDiscoveredURLs[page.url]={"title":contentTitle,"content":contentTxt}
                                        print("***Added URL:", page.url)

                                    else:
                                        print("cannot connect to URL:"+URL)
                                else:
                                    print("Potential New Discovered URL found in Store (not adding):   "+currLink)

                                print("\n\nCurrently at Main Seed (Depth 2):"+str(indexCurr+1)," of ",len(lister),"\n| Main Seed Link:"+URL)  
                        
                        else:
                            print("cannot connect to URL:"+URL)
                    else:
                        print("duplicated URL found in Store (not adding):   ",URL)

                print("\n\nCurrently at Main Seed (Depth 1):"+str(indexCurr+1)," of ",len(lister),"\n| Main Seed Link:"+URL)     
            
            else:
                print("cannot connect to URL:"+URL)            
        print("\n\nCurrently at Main Seed (Depth 1):"+str(indexCurr+1)," of ",len(lister),"\n| Main Seed Link:"+URL)
    
    #Finished Main Seeds (addedJson)
    if len(addedJson.keys())!=0:
        wikiArray.append(addedJson)
    
    #Finished 2nd Deep Link Crawl
    if len(new1stDiscoveredURLs.keys())!=0:
        wikiArray.append(new1stDiscoveredURLs)
    
    #Finished 3rd Deep Link Crawl
    if len(new2ndDiscoveredURLs.keys())!=0:
        wikiArray.append(new1stDiscoveredURLs)
    
    #Export Json to File
    try:
        wikiJson ={}
        dict1={}

        for item in wikiArray:
            for key, value in item.items():
                wikiJson[key]=value
        dict1["all_content"]=wikiJson
        with open('wikiFile.json', 'w') as outfile:
            json.dump(dict1, outfile)
        print("Exported to JSON for Wiki URLs as name: \'wikiFile.json\'")
    except Exception as e:
        print("Couldn't export to CSV for Wiki URLs... Stack Trace Here:\n")
        print(e)


    

#start with empty dictionary {}
newArray =[]
getLinksFromSearch(lister,len(lister)-1)


#Crawler Flow:
'''
    Go through List of keywords>get URL from Wiki Search> EITHER: 
    ##########(1) 
    [[Depth 1]]
    Wiki Search Returns Wiki Page (Not Search Page):
    Crawl Wiki Page for Title and Content>

    [[Depth 2]]
    Get newly Discovered Links in Wiki Page (known as '2nd Deep Layer')>
    Crawl Wiki Page of newly Discovered Links (in '2nd Deep Layer') for Title and Content>

    ##########[[END for (1)]]




    ##########(2) 
    Wiki Search Returns Wiki Search Page LIST
    [[Depth 1]]
    Wiki Search Returns Wiki Page (Not Search Page):
    Crawl Wiki Page for Title and Content>

    [[Depth 2]]
    Get newly Discovered Links in Wiki Page (known as '2nd Deep Layer')>
    Crawl Wiki Page of newly Discovered Links (in '2nd Deep Layer') for Title and Content>
    ##########[[END for (2)]]

'''



Currently Main Seed:1  of  21699


zzzzz Sleeping for  2  sec(s)
ooooo Awake Now
Page is Wiki Search Page: https://en.wikipedia.org/w/index.php?search=Glasgow+Kingdom&ns0=1


zzzzz Sleeping for  2  sec(s)
ooooo Awake Now
URL: https://en.wikipedia.org/wiki/Glasgow
TITLE~
Glasgow
content~
Largest city in Scotland
This article is about the city in Scotland. For other uses, see Glasgow (disambiguation).


City and council area in ScotlandGlasgow
Glesca, also Glesga (Scots)[1]Glaschu (Scottish Gaelic)City and council areaFrom top, left to right: Skyline of Glasgow City Centre, Tolbooth Steeple, People Make Glasgow signage at the former City of Glasgow College building, The Glasgow Tower, tallest structure in Scotland and tallest structure that can rotate 360°, the OVO Hydro, SEC Armadillo and the Finnieston Crane, George Square, Cineworld Glasgow, the tallest cinema in the world

Coat of armsNicknames: "Glesca/Glesga", "The Dear Green Place", "Baile Mòr nan Gàidheal"[2]Motto(s): Let Glasg

ooooo Awake Now
URL: https://en.wikipedia.org/wiki/Glasgow_(disambiguation)
TITLE~
Glasgow (disambiguation)
content~



Look up Glasgow in Wiktionary, the free dictionary.

Glasgow is the largest city in Scotland. 
Glasgow may also refer to:

Greater Glasgow Metropolitan Area
Glasgow (Scottish Parliament electoral region), an electoral region in the Scottish Parliament
University of Glasgow
Glasgow Airport
Glasgow Prestwick Airport
Glasgow Subway
Contents

1 Places

1.1 Canada
1.2 New Zealand
1.3 Suriname
1.4 United States


2 Sports
3 Transportation
4 Other uses
5 See also



Places[edit]
Canada[edit]
Glasgow, Ontario, a community located in the Regional Municipality of Durham
Glasgow, a community  in Caledon, Ontario
New Zealand[edit]
Glasgow Range, a mountain range on the South Island
Suriname[edit]
Glasgow, Suriname, in Nickerie District
United States[edit]
Glasgow, Alabama, a place in Alabama
Glasgow, California, a place in California
Glasgo, Connecticut
Glasgow, Delaware
Glasgow,

ooooo Awake Now
URL: https://en.wikipedia.org/wiki/Scots_language
TITLE~
Scots language
content~
Germanic language
For the Celtic language, see Scottish Gaelic.Not to be confused with Scottish English.


ScotsLowland Scots, Broad Scots(Braid) Scots, Lallans, DoricNative toUnited Kingdom, Republic of IrelandRegion
Scotland: Scottish Lowlands, Northern Isles, Caithness, Arran and Campbeltown
Ulster (Ireland): Counties Down, Antrim, Londonderry, Donegal and Armagh
EthnicityLowland Scots peopleNative speakersNumbers disputed. 99,200 (2019)[1]In 2011, 1,541,693 people in Scotland alone reported speaking Scots.[2]Language familyIndo-European

GermanicWest GermanicNorth Sea GermanicAnglo-FrisianAnglicScotsEarly formsNorthumbrian Old English
Early Middle English
Early Scots
Middle Scots



Dialects
Central
Southern
Ulster
Northern
Insular
Writing systemLatinOfficial statusOfficial language inScotland[3]Recognised minoritylanguage inNorthern Ireland (as Ulster Scots), Republic of Ireland (Count

ooooo Awake Now
URL: https://en.wikipedia.org/wiki/Scottish_Gaelic_language
TITLE~
Scottish Gaelic
content~
For the Germanic language that diverged from Middle English, see Scots language.
Goidelic Celtic language of Scotland


Scottish GaelicScots Gaelic, GaelicGàidhligPronunciation[ˈkaːlɪkʲ]Native toUnited Kingdom, CanadaRegionScotland; Cape Breton Island, Nova ScotiaEthnicityScottish peopleNative speakers57,000 fluent L1 and L2 speakers in Scotland[1] (2011)87,000 people in Scotland reported having some Gaelic language ability in 2011;[1] 1,300 fluent in Nova Scotia[2]Language familyIndo-European

CelticInsular CelticGoidelicScottish GaelicEarly formsPrimitive Irish
Old Irish
Middle Irish


Dialects
Canadian
Mid-Minch
East Sutherland
Galwegian †
many others
Writing systemLatin (Scottish Gaelic alphabet)Official statusOfficial language inScotland[3]Recognised minoritylanguage inNova Scotia, CanadaLanguage codesISO 639-1gdISO 639-2glaISO 639-3glaGlottologscot1245ELPScottish GaelicLing

ooooo Awake Now
URL: https://en.wikipedia.org/wiki/City_status_in_the_United_Kingdom
TITLE~
City status in the United Kingdom
content~
Status granted by royal charter to settlements in the United Kingdom
"Cathedral city" redirects here. For other uses, see Cathedral city (disambiguation).


 Until the 19th century, city status in England and Wales was associated with the presence of a cathedral, such as York Minster.
City status in the United Kingdom is granted by the monarch of the United Kingdom to a select group of communities. As of 26 January 2022[update],[1][2] there are 70 cities in the United Kingdom—52 in England,[3][4] six in Wales, seven in Scotland, and five in Northern Ireland.[5] Although it carries no special rights, the status of city can be a marker of prestige and confer local pride.[6]
The status does not apply automatically on the basis of any particular criterion, though in England and Wales it was traditionally given to towns with diocesan cathedrals. This associa

...

ooooo Awake Now
URL: https://en.wikipedia.org/wiki/Glasgow_Tolbooth
TITLE~
Glasgow Tolbooth
content~
Municipal Building in Scotland


Glasgow TolboothGlasgow TolboothLocationGlasgow Cross, GlasgowCoordinates55°51′24″N 4°14′37″W﻿ / ﻿55.8568°N 4.2436°W﻿ / 55.8568; -4.2436Coordinates: 55°51′24″N 4°14′37″W﻿ / ﻿55.8568°N 4.2436°W﻿ / 55.8568; -4.2436Built1634ArchitectJohn BoydArchitectural style(s)Scottish baronial style
Listed Building – Category AOfficial nameHigh Street, Glasgow Cross, the Tolbooth SteepleDesignated6 July 1966Reference no.LB32717

Shown in Glasgow
The Glasgow Tollbooth was a municipal structure at Glasgow Cross, Glasgow, Scotland. The main block, which was the meeting place of the Royal Burgh of Glasgow, was demolished in 1921 leaving only the steeple standing. The steeple is a Category A listed building.[1]

History[edit]
 The surviving steeple
The first tolbooth erected on the site at Glasgow Cross dated back at least to the mid-14th century.[2][3] After it became ruino

ooooo Awake Now
URL: https://en.wikipedia.org/wiki/City_of_Glasgow_College
TITLE~
City of Glasgow College
content~
College in Glasgow City, Scotland, UK


This article needs additional citations for verification. Please help improve this article by adding citations to reliable sources. Unsourced material may be challenged and removed.Find sources: "City of Glasgow College" – news · newspapers · books · scholar · JSTOR (August 2019) (Learn how and when to remove this template message)
City of Glasgow CollegeColaiste Baile GlaschuCeety o Glesga CollegeRiverside Campus on the River ClydeOther namesCoGCScotland's Super CollegeFormer namesCentral College, Glasgow Glasgow Metropolitan College  Glasgow College of Nautical StudiesMottoLet Learning FlourishTypeFurther and Higher EducationEstablishedSeptember 2010ChairmanAlisdair BarronPrincipalPaul LittleStudents27,000LocationGlasgow, ScotlandColours   Purple and silverWebsitewww.cityofglasgowcollege.ac.uk
Coordinates: 55°51′47″N 4°14′38″W﻿ / ﻿

ooooo Awake Now
URL: https://en.wikipedia.org/wiki/Glasgow_Tower
TITLE~
Glasgow Tower
content~
Observation tower in Glasgow City, Scotland, UK


Glasgow Tower  Millenium Tower (2001)Record heightTallest in Scotland since 2001[I]Preceded byBluevale and Whitevale TowersGeneral informationTypeObservation towerLocationGlasgow, ScotlandCoordinates55°51′33″N 4°17′46″W﻿ / ﻿55.8592°N 4.2961°W﻿ / 55.8592; -4.2961Coordinates: 55°51′33″N 4°17′46″W﻿ / ﻿55.8592°N 4.2961°W﻿ / 55.8592; -4.2961Completed2001HeightAntenna spire127 m (417 ft)Design and constructionArchitectRichard HordenStructural engineerBuro HappoldAwards and prizesHolds a Guinness World Record for being the tallest tower in the world in which the whole structure is capable of rotating 360 degrees.[1]Websitewww.glasgowsciencecentre.org/discover/our-experiences/glasgow-tower
Glasgow Tower (formerly the Millennium Tower)[2] is a 127 metres (417 ft) free-standing landmark observation tower located on the south bank of the River Clyde in G

ooooo Awake Now
URL: https://en.wikipedia.org/wiki/OVO_Hydro
TITLE~
OVO Hydro
content~
Multipurpose indoor arena in Glasgow, Scotland
"The Hydro" redirects here. For the Tasmanian electrical power authority known by that name, see Hydro Tasmania.


The OVO HydroThe HydroFormer namesScotland's National Arena (planning)Scottish Hydro Arena (construction) The SSE Hydro (September 30, 2013 – October 2021)AddressExhibition WaySECGlasgowG3 8YWLocationGlasgow, ScotlandCoordinates55°51′37″N 4°17′06″W﻿ / ﻿55.860297°N 4.284926°W﻿ / 55.860297; -4.284926Coordinates: 55°51′37″N 4°17′06″W﻿ / ﻿55.860297°N 4.284926°W﻿ / 55.860297; -4.284926Public transitExhibition Centre railway stationOwnerSEC LimitedOperatorAEG Live / ASM GlobalCapacity12,306 (all seated)14,500 (with standing)ConstructionBroke groundFebruary 2011Built2011–2013Opened30 September 2013 (2013-09-30)Construction cost£125 millionArchitectFoster + PartnersProject managerTurner & TownsendStructural engineerArupGeneral contractorLendleaseWeb

ooooo Awake Now
URL: https://en.wikipedia.org/wiki/SEC_Armadillo
TITLE~
SEC Armadillo
content~
Auditorium located in Glasgow, Scotland


This article needs additional citations for verification. Please help improve this article by adding citations to reliable sources. Unsourced material may be challenged and removed.Find sources: "SEC Armadillo" – news · newspapers · books · scholar · JSTOR (April 2019) (Learn how and when to remove this template message)
SEC ArmadilloThe 2k ArmadilloClyde Auditorium at night, May 2008Former namesClyde Auditorium (2000–2017)AddressFinnieston StreetGlasgowG3 8YWScotlandLocationScottish Event CampusCoordinates55°51′34″N 4°17′17″W﻿ / ﻿55.859496°N 4.287962°W﻿ / 55.859496; -4.287962Coordinates: 55°51′34″N 4°17′17″W﻿ / ﻿55.859496°N 4.287962°W﻿ / 55.859496; -4.287962Public transitExhibition Centre railway stationOwnerSEC LimitedCapacity3,000ConstructionBroke groundSeptember 1995; 26 years ago (1995-09)BuiltSeptember 1995; 26 years ago (1995-09)to August 1997;

# [Wikipedia Crawler End]

# Backup code (Old)

In [ ]:
import requests
import time
import json
from bs4 import BeautifulSoup
from itertools import combinations
import time

words2Scrap=["Computer", "Glasgow", "United", "Kingdom", "Library", "Fog", "Empires", "Doctor", "Hospital", "Bachelor", 
"Degree", "Internet", "Things", "Information", "Info", "Retrieval", "Retrieve", "Info", "Universe", "University"]

#Source: https://stackoverflow.com/questions/464864/how-to-get-all-possible-combinations-of-a-list-s-elements#answer-32555776
def combs(x):
    return [c for i in range(len(x)+1) for c in combinations(x,i)]


lister1= list(combs(words2Scrap))
lister = []

for item in lister1:
    if (len(item) == 5 or len(item) < 6) and len(item) !=0 :
        lister.append(item)

def sleepingNow(delayTime):
    print("\n\nzzzzz Sleeping for ",delayTime," sec(s)")
    time.sleep(delayTime)
    print("ooooo Awake Now")

def formatSearchTerm4url(typle,index):
    searchTerm=""
    for i,d in enumerate(list(typle[index])):
        if (i!=0):
            searchTerm+="+"+str(d)
        else:
            searchTerm=d
    urlMain= f"https://en.wikipedia.org/w/index.php?search="+ searchTerm

    # get the redirected URL's destination (.url) A.K.A non-Search query URL
    urlMain = requests.get(urlMain).url
    return urlMain
new1stDiscoveredURLs={}
new2ndDiscoveredURLs={}
wikiArray=[]
addedJson={}

def getLinksFromSearch(lister,index):

    
    for indexCurr,itemCurr in enumerate(lister):
        
        URL = formatSearchTerm4url(lister,indexCurr)
        print("\n\nCurrently Main Seed:"+str(indexCurr+1)," of ",len(lister))
        if (indexCurr<0):
            pass
        elif (indexCurr>2):
            break
        elif (indexCurr<index+1):
            
            #if using Keyword Search
            sleepingNow(5)
            connectPassed=True
            try:
                page = requests.get(URL)
            except:
                connectPassed=False
            

            if (connectPassed == True):
                soup = BeautifulSoup(page.content, "html.parser")
                job_elements = soup.find("h1",{"id":"firstHeading"})
                contentTitle = job_elements.text

                if ("Search results" in job_elements.text):
                    pass
                
                
                
                
                    print("Page is Wiki Search Page: "+URL)
                    #returned page is a search result page with multiple wiki pages
                    job_elements = soup.find("ul",{"class":"mw-search-results"}).find_all("a",href=True)

                    for index1,item1 in enumerate(job_elements):
                        url = "https://en.wikipedia.org"+item1['href']

                        found=False
                        for index,item in enumerate(wikiArray):
                            if (url in item or url in addedJson):
                                found = True

                        if (found==False):
                            sleepingNow(5)
                            contentTxt = ""
                            
                            #if using Keyword Search
                            sleepingNow(5)
                            connectPassed=True
                            try:
                                page = requests.get(URL)
                            except:
                                connectPassed=False


                            if (connectPassed == True):
                                page = requests.get(url)
                                soup = BeautifulSoup(page.content, "html.parser")
                                job_elements = soup.find("h1",{"id":"firstHeading"})
                                contentTitle = job_elements.text
                                job_elements = soup.find("div",{"id":"mw-content-text"}).find_all()
                                for item in job_elements:
                                    try:
                                        contentTxt = contentTxt + item.text
                                    except:
                                        pass
                                print("URL: "+page.url)
                                print("TITLE~\n"+contentTitle)
                                print("content~\n"+contentTxt)

                                addedJson[page.url]={"title":contentTitle,"content":contentTxt}
                                print("***Added URL:", page.url)
                            else:
                                print("cannot connect to URL:"+URL)
                        else:
                            print("duplicated URL found in Store (not adding):   ",url)

                        print("\n\nCurrently at Main Seed:"+str(indexCurr+1)," of ",len(lister),"\n| Main Seed Link:"+URL)
    
    
                else:
                    print("Page is Wiki Details Page: "+URL)

                    found=False
                    for index,item in enumerate(wikiArray):
                        if (URL in item or URL in addedJson):
                            found = True

                    if (found==False):
                        contentTxt = ""
                        
                        #if using Keyword Search
                        sleepingNow(5)
                        connectPassed=True
                        try:
                            page = requests.get(URL)
                        except:
                            connectPassed=False


                        if (connectPassed == True):
                            soup = BeautifulSoup(page.content, "html.parser")
                            job_elements = soup.find("h1",{"id":"firstHeading"})
                            contentTitle = job_elements.text
                            job_elements = soup.find("div",{"id":"mw-content-text"}).find_all()
                            for item in job_elements:
                                try:
                                    contentTxt = contentTxt + item.text
                                except:
                                    pass

                            print("URL: "+page.url)
                            print("TITLE~\n"+contentTitle)
                            print("content~\n"+contentTxt)

                            addedJson[page.url]={"title":contentTitle,"content":contentTxt}
                            print("***Added URL:", page.url)

                            ####################################
                            #Get 2nd Deep Link from Main Seed
                            potentialLinks = soup.find("div",{"id":"mw-content-text"}).find_all("a", href=True)
                            for link in potentialLinks:
                                currLink = link['href']
                                failed = False

        #                         try:
        #                             currLink = currLink.split(".")[0]
        #                         except:
        #                             pass

                                if currLink[0] == "#":
                                    failed = True


                                #if URL is not a reference within the Current Wiki Page
                                if (failed == False):
                                    #Check if URL is in Store
                                    for index,item in enumerate(wikiArray):
                                        if (currLink in addedJson):
                                            failed = True
                                        jsonData = wikiArray[index]
                                        for key1 in jsonData.keys():
                                            if key1 in currLink:
                                                failed = True
                                if ("/wiki/" in currLink and "wikipedia.org" not in currLink):
                                    currLink = "https://en.wikipedia.org" + currLink

                                if (("search=" in currLink or "wiki/File:" in currLink) and "wikipedia" in currLink):
                                    print("Potential New Discovered URL is a Wiki File/Search (not adding):   "+currLink)
                                elif (failed==False and "wikipedia" in currLink):
                                    contentTxt = ""
                                    
                                    #if using Keyword Search
                                    sleepingNow(5)
                                    connectPassed=True
                                    try:
                                        page = requests.get(currLink)
                                    except:
                                        connectPassed=False

                                    if (connectPassed == True):
                                        soup = BeautifulSoup(page.content, "html.parser")
                                        job_elements = soup.find("h1",{"id":"firstHeading"})
                                        contentTitle = job_elements.text
                                        job_elements = soup.find("div",{"id":"mw-content-text"}).find_all()
                                        for item in job_elements:
                                            try:
                                                contentTxt = contentTxt + item.text
                                            except:
                                                pass

                                        print("URL: "+page.url)
                                        print("TITLE~\n"+contentTitle)
                                        print("content~\n"+contentTxt)

                                        new1stDiscoveredURLs[page.url]={"title":contentTitle,"content":contentTxt}
                                        print("***Added URL:", page.url)

                                        ####################################
                                        #Get 3rd Deep Link from Main Seed
#                                         potentialLinks = soup.find("div",{"id":"mw-content-text"}).find_all("a", href=True)
#                                         for link in potentialLinks:
#                                             currLink = link['href']
#                                             failed = False

#                                             if currLink[0] == "#":
#                                                 failed = True

#                                             #if URL is not a reference within the Current Wiki Page
#                                             if (failed == False):
#                                                 #Check if URL is in Store
#                                                 for index,item in enumerate(wikiArray):
#                                                     if (currLink in addedJson):
#                                                         failed = True
#                                                     if (currLink in new1stDiscoveredURLs):
#                                                         failed = True
#                                                     jsonData = wikiArray[index]
#                                                     for key1 in jsonData.keys():
#                                                         if key1 in currLink:
#                                                             failed = True

#                                             if ("/wiki/" in currLink and "wikipedia.org" not in currLink):
#                                                 currLink = "https://en.wikipedia.org" + currLink

#                                             if (("search=" in currLink or "wiki/File:" in currLink) and "wikipedia" in currLink):
#                                                 print("Potential New Discovered URL is a Wiki File/Search (not adding):   "+currLink)
#                                             elif (failed==False and "wikipedia" in currLink):
#                                                 contentTxt = ""
                                                
#                                                 #if using Keyword Search
#                                                 sleepingNow(5)
#                                                 connectPassed=True
#                                                 try:
#                                                     page = requests.get(currLink)
#                                                 except:
#                                                     connectPassed=False
#                                                 if (connectPassed == True):
#                                                     soup = BeautifulSoup(page.content, "html.parser")
#                                                     job_elements = soup.find("h1",{"id":"firstHeading"})
#                                                     contentTitle = job_elements.text
#                                                     job_elements = soup.find("div",{"id":"mw-content-text"}).find_all()
#                                                     for item in job_elements:
#                                                         try:
#                                                             contentTxt = contentTxt + item.text
#                                                         except:
#                                                             pass

#                                                     print("URL: "+page.url)
#                                                     print("TITLE~\n"+contentTitle)
#                                                     print("content~\n"+contentTxt)

#                                                     new2ndDiscoveredURLs[page.url]={"title":contentTitle,"content":contentTxt}
#                                                     print("***Added URL:", page.url)
                                                
#                                                 else:
#                                                     print("cannot connect to URL:"+URL)
#                                             else:
#                                                 print("Potential New Discovered URL found in Store (not adding):   "+currLink)

#                                             print("\n\nCurrently at Main Seed (Depth 3):"+str(indexCurr+1)," of ",len(lister),"\n| Main Seed Link:"+URL)  
                                    
                                    else:
                                        print("cannot connect to URL:"+URL)
                                else:
                                    print("Potential New Discovered URL found in Store (not adding):   "+currLink)

                                print("\n\nCurrently at Main Seed (Depth 2):"+str(indexCurr+1)," of ",len(lister),"\n| Main Seed Link:"+URL)  
                        
                        else:
                            print("cannot connect to URL:"+URL)
                    else:
                        print("duplicated URL found in Store (not adding):   ",URL)

                print("\n\nCurrently at Main Seed (Depth 1):"+str(indexCurr+1)," of ",len(lister),"\n| Main Seed Link:"+URL)     
            
            else:
                print("cannot connect to URL:"+URL)            
        print("\n\nCurrently at Main Seed (Depth 1):"+str(indexCurr+1)," of ",len(lister),"\n| Main Seed Link:"+URL)
    
    
    #Finished Main Seeds (addedJson)
    if len(addedJson.keys())!=0:
        wikiArray.append(addedJson)
    
    #Finished 2nd Deep Link Crawl
    if len(new1stDiscoveredURLs.keys())!=0:
        wikiArray.append(new1stDiscoveredURLs)
    
    #Finished 3rd Deep Link Crawl
    if len(new2ndDiscoveredURLs.keys())!=0:
        wikiArray.append(new1stDiscoveredURLs)
    
    #Export Json to File
    try:
        wikiJson ={}
        dict1={}

        for item in wikiArray:
            for key, value in item.items():
                wikiJson[key]=value
        dict1["all_content"]=wikiJson
        with open('wikiFile.json', 'w') as outfile:
            json.dump(dict1, outfile)
        print("Exported to JSON for Wiki URLs as name: \'wikiFile.json\'")
    except Exception as e:
        print("Couldn't export to CSV for Wiki URLs... Stack Trace Here:\n")
        print(e)

    

#start with empty dictionary {}
newArray =[]
getLinksFromSearch(lister,len(lister)-1)


#Flow:
'''
Go through List of keywords>get URL from Wiki Search> EITHER: 
##########(1) 
[[Depth 1]]
Wiki Search Returns Wiki Page (Not Search Page):
Crawl Wiki Page for Title and Content>

[[Depth 2]]
Get newly Discovered Links in Wiki Page (known as '2nd Deep Layer')>
Crawl Wiki Page of newly Discovered Links (in '2nd Deep Layer') for Title and Content>

##########[[END for (1)]]




##########(2) 
Wiki Search Returns Wiki Search Page LIST
[[Depth 1]]
Wiki Search Returns Wiki Page (Not Search Page):
Crawl Wiki Page for Title and Content>

[[Depth 2]]
Get newly Discovered Links in Wiki Page (known as '2nd Deep Layer')>
Crawl Wiki Page of newly Discovered Links (in '2nd Deep Layer') for Title and Content>



##########[[END for (2)]]

'''


#[[Depth 3]]
# Get newly Discovered Links in Wiki Page (known as '3rd Deep Layer')>
# Crawl Wiki Page of newly Discovered Links (in '3rd Deep Layer') for Title and Content



Currently Main Seed:1  of  21699


zzzzz Sleeping for  5  sec(s)
ooooo Awake Now
Page is Wiki Details Page: https://en.wikipedia.org/wiki/Computer


zzzzz Sleeping for  5  sec(s)


In [73]:
def getLinksFromSearch(lister,index,timeArray,wikiArray,depth):

    if depth ==0:
        addedJson={}
        for indexCurr,item in enumerate(lister):
            print("Current Index:"+str(indexCurr))
            if (indexCurr <21): #== 0
                pass
            elif (indexCurr<index+1):

                URL = formatSearchTerm4url(lister,indexCurr)

                #if using Keyword Search
                if(type(URL)==str):
                    sleepingNow(5)
                    page = requests.get(URL)

                    soup = BeautifulSoup(page.content, "html.parser")
                    job_elements = soup.find("h1",{"id":"firstHeading"})

                    if ("Search results" in job_elements.text):
                        #returned page is a search result page with multiple wiki pages
                        job_elements = soup.find("ul",{"class":"mw-search-results"}).find_all("a",href=True)
                        for index,item in enumerate(job_elements):

                            found=False
                            
                            url = "https://en.wikipedia.org"+item['href'] #####################
                            for index,item in enumerate(wikiArray):
                                if (url in item):
                                    found = True

                            if (found==False and len(wikiArray) == 0):
                                addedJson[url]={"title":"","content":""}
                                print("***Added URL:",url)
                            else:
                                print("duplicated URL (not adding):   ",url)
                        if len(addedJson.keys())!=0:
                            wikiArray.append(addedJson)
                            print("Total No. of URLs at Depth No. "+str(depth)+": ",str(len(wikiArray[len(wikiArray)-1].keys())))
                        else:
                            print("Total No. of URLs at Depth No. "+str(depth)+": 0")
                        print("URL "+str(indexCurr)+" of "+str(len(lister))+"\n | Link:"+item)

                    else:
                        print
                        contents = soup.find("div",{"id":"content"})
                        contentSoup = contents.find("div",{"id":"mw-content-text"}).find_all(href=True)
                        for index,item in enumerate(contentSoup):

                            #if href (URL) already well-formatted with HTTP(S)
                            if ("http" not in item["href"]):
                                hrefCase = "https://en.wikipedia.org"+item["href"]
                            elif (("wikipedia" not in item["href"]) or ("title=Template" in item["href"]) or ("&action=edit" in item["href"])):
                                pass #wiki not and title=Template/@action=edit
                            else:
                                hrefCase = item['href']

                            toCheck =[URL,hrefCase]
                            for url in toCheck:
                                if ("http" not in url):
                                    url = "https://en.wikipedia.org"+url
                                #if URL is a file with extension type '.' or not in dictionary
                                if (("." not in url.split("/")[-1]) and (url not in wikiArray)):
                                    print("not image/ wikiArray 1")

                                    #if referencing to a section in current wiki page, then ignore (not add to URL Frontier)
                                    if ((url[0] == "#") or ("mw-data:TemplateStyles" in hrefCase)):
                                        pass

                                    #if no redirect to same wiki page but different section
                                    #check if href base URL already in dictionary
                                    else:

                                        #if potential # in URL, investigate
                                        if ("#" in url):
                                            #if base url is not in addedJson, add into Dict
                                            if (url.split("#")[0] not in addedJson):
                                                found = False
                                                #not found in wikiArray
                                                for index,item in enumerate(wikiArray):
                                                    if (url.split("#")[0] in item):
                                                        found = True

                                                if (found==False):
                                                    addedJson[url.split("#")[0]]={"title":"","content":""}
                                                    print("***Added URL:",url.split("#")[0])

                                            #if duplicated URL, don't add into Dict
                                            else:
                                                print("duplicated URL (not adding):   ",url)
                                                pass


                                        else:
                                            #if base url is not in addedJson
                                            if (url not in addedJson):
                                                found = False

                                                #not found in wikiArray
                                                for index,item in enumerate(wikiArray):
                                                    if (url in item):
                                                        found = True

                                                #add into Dict
                                                if (found==False):
                                                    addedJson[url]={"title":"","content":""}
                                                    print("***Added URL:",url)


                                                #if duplicated URL, don't add into Dict
                                                else:
                                                    print("duplicated URL (not adding):   ",url)
                                                    pass
                                            #if duplicated URL, don't add into Dict
                                            else:
                                                print("duplicated URL (not adding):   ",url)
                                                pass

                        wikiArray.append(addedJson)
                        print("URL "+str(indexCurr)+" of "+str(len(lister))+"\n | Link:"+item)
                        print("Total No. of URLs at Depth No. "+str(depth)+": ",str(len(wikiArray[len(wikiArray)-1].keys())))


            else:

                try:
                    wikiJson ={}
                    dict1={}

                    for item in wikiArray:
                        for key, value in item.items():
                            wikiJson[key]=value
                    dict1["all_content"]=wikiJson
                    with open('wikiFile.json', 'w') as outfile:
                        json.dump(dict1, outfile)
                except Exception as e:
                    print("Couldn't export to CSV for Wiki URLs... Stack Trace Here:\n")
                    print(e)

#start with empty dictionary {}
newArray =[]
getLinksFromSearch(lister,len(lister)-1,[10*60,time.time()], newArray,1)